In [2]:
import os
import re

import numpy as np

from collections import defaultdict

os.chdir('/home/reese56/w266_final/scripts/assembly/')
from subject import subject_keywords

from constant import DOC_SAMPLE_PATH, DOCUMENT, DOC_ALL_PATH
from subject import subject_keywords
from document import load_documents

In [3]:
PROJ_PATH_REESE = '/home/reese56/w266_final/'
PROJ_PATH_ROWAN = '/home/rowan/w266_final/'
# DOC_PATH = os.path.join(PROJ_PATH_REESE,'data/gen-data/doc/%s/documents_%s.txt')

In [4]:
num_samp = len(subject_keywords.keys())*20
sessions = np.random.randint(43,112, num_samp)
subjects = list(subject_keywords.keys())
sessions = np.arange(43,112)

In [5]:
documents_df = load_documents(sessions, DOC_ALL_PATH)

In [10]:
rand_sample_l = {s:documents_df[documents_df['subject'] == s].sample(20, axis = 0) for s in subjects}
rand_sample_l['abortion']

,speakerid,lastname,firstname,chamber,state,gender,party,document,subject,session
163962,104109641,PRESSLER,LARRY,S,SD,M,R,for education training and support services if...,abortion,104
150593,95107970,ST. GERMAIN,FERNAND,H,RI,M,D,first months of have already exceeded milli...,abortion,95
149402,100113200,MOAKLEY,JOHN,H,MA,M,D,religious belief must always be compromised th...,abortion,100
127187,108113921,WARNER,JOHN,S,VA,M,R,air emissions endangers our health and environ...,abortion,108
142013,103113390,TRAFICANT,JAMES,H,OH,M,D,is to guarantee that the water from our taps i...,abortion,103
147693,94109271,JOHNSTON,JOHN,S,LA,M,D,inc east th street ny e a iron ore lessors...,abortion,94
139751,97120181,LEAHY,PATRICK,S,VT,M,D,tripartite system of government we in the sena...,abortion,97
129375,105113210,MORELLA,CONSTANCE,H,MD,F,R,and independent judgment have always been diff...,abortion,105
114811,87097220,GRANAHAN,KATHRYN,H,PA,F,D,improvement of defense department security met...,abortion,87
154975,99113020,GILMAN,BENJAMIN,H,NY,M,R,operates that we have to be here but it should...,abortion,99


In [136]:

class DocumentSubjectEvaluator(object):
    
    def __init__(self, df):
        """
        Args:
        - df : (DataFrame) a document dataframe
        """
        
        'Initialization'
        self.df = df.reset_index(drop = True)
        self.subjects = df.subject.unique().tolist()
        self.row_scores = []
        self.false_keys = []
        self.cross_keys = []
    


    def subjectKeywordSearch(r, subject):

        found = []
        for k in subject_keywords[subject]:
            if bool(re.search(r'\b' + k, r.document)):
                found.append(k)
        if len(found)>0:
            print('  ',subject, '-', ', '.join(found))

    def keywordSearch(r):

        for s in subject_keywords.keys():
            subjectKeywordSearch(r,s)

    def examine_document(r):

        print(100*"=", "\n")
        print("PARTY:", r.party)
        print("GENDER:", r.gender)
        print("STATE:", r.state)
        print("CHAMBER:", r.chamber)
        print("SESSION:", r.session,'\n')
        print('KEYWORDS:')
        keywordSearch(r)
        print("\n")
        print(r.document)
        print("\n")


    def evaluate_subject(subject):

        for r in df.itertuples():
            examine_document(r)
            comp = input('Subject Comprehension (True/1 or False/0):')
            self.row_scores.append(comp)
            if not comp:
                sub_key = input("Faulty Subject Keywords:").split(',')
                for sk in sub_key:
                    self.fasle_keys.append(sk)
            cross_sub = input("Cross Subjects:").split(',')
            if not (cross_sub[0] == ''):
                for csk in cross_sub:
                    csub_key = input("Faulty Keywords:").split('')

            yield

In [137]:
abortion = evaluate_documents(rand_sample_l, 'abortion')

In [138]:
next(abortion)

NameError: name 'fcs_keys' is not defined

In [75]:
# keywordSearch(rand_sample_l['abortion'], subject_keywords['abortion'][5])

In [56]:
abortion = examine_subject(rand_sample_l, 'abortion')

In [57]:
next(abortion)


party, R
for education training and support services if we want poor parents to work they need these services they need child care and transportation for example the goals of the bill include preventing teen pregnancy and outofwedlock births unfortunately and incredibly family planning services the key to reducing outofwedlock births the vast majority of which are unintended are not even mentioned in this bill which does away with the yearold requirement that states offer family planning services to all afdc recipients meanwhile in just the




In [6]:
def load_subdoc(doc_set, subject):
    
    with open(DOC_PATH % (doc_set, subject)) as f:
        subject_doc = [l for l in f]
    
    return subject_doc


def false_match_keyword(keyword, subject_doc):
    
    false_matches = defaultdict(lambda : 0)
    
    for l in subject_doc:
        match = re.search(r'\s\w+%s\w+\s' % keyword, l)
        try:
            match_str = match[0][1:-1]
        except:
            continue
        false_matches[match_str] += 1
    
    return false_matches


def subject_false_match(subject_doc):
    
    false_match_dict = defaultdict(list)
    sub = re.search(r'\|\w*?$', subject_doc[1])[0][1:]
    
    for keyword in subject_keywords[sub]:
        fmd = false_match_keyword(keyword, subject_doc)
        if fmd:
            false_match_dict[keyword] = fmd
    
    return false_match_dict
    

In [ ]:
defense_doc = load_subdoc('doc1','defense')
defense_false = subject_false_match(defense_doc)

In [ ]:
totals = defaultdict(lambda : 0)
for key, val in defense_false.items():
     totals[key] = sum(val.values())

In [ ]:
defense_false

In [ ]:
sum(totals.values())/len(defense_doc)

In [ ]:
defense_doc[10]

In [ ]:
subject_keywords['defense']